In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import numpy as np
import random
from PIL import Image, ImageEnhance
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Flatten, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model, Model

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf

data_dir = '/content/drive/MyDrive/archive'
train_dir = os.path.join(data_dir, "Training")
test_dir = os.path.join(data_dir, "Testing")

IMAGE_SIZE = 128
BATCH_SIZE = 64

# مرحله ۱: جمع‌آوری مسیر همه تصاویر و برچسب‌ها
all_image_paths, all_labels = [], []
for label in os.listdir(train_dir):
    folder = os.path.join(train_dir, label)
    if not os.path.isdir(folder):
        continue
    for image_name in os.listdir(folder):
        if image_name.lower().endswith(('.jpg', '.png', '.jpeg')):
            all_image_paths.append(os.path.join(folder, image_name))
            all_labels.append(label)

df = pd.DataFrame({'path': all_image_paths, 'label': all_labels})
print(f" قبل از پاک‌سازی: {len(df)}")

# مرحله ۲: حذف داده‌های تکراری و گمشده
df.drop_duplicates(subset='path', inplace=True)
df.dropna(inplace=True)
df = df[df['path'].apply(lambda x: os.path.exists(x))]
print(f" بعد از پاک‌سازی: {len(df)}")

#  مرحله ۳: Stratified train/test Split
train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df['label'],
    random_state=42)

print(f" Train: {len(train_df)} | Test: {len(test_df)}")

# مرحله ۴: tf.data.Dataset
def preprocess_image(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [IMAGE_SIZE, IMAGE_SIZE])
    img = img / 255.0
    return img, label

label_to_index = {label: idx for idx, label in enumerate(sorted(df['label'].unique()))}
train_df['label_idx'] = train_df['label'].map(label_to_index)
test_df['label_idx'] = test_df['label'].map(label_to_index)

train_paths = train_df['path'].values
train_labels = tf.keras.utils.to_categorical(train_df['label_idx'].values, num_classes=len(label_to_index))
test_paths = test_df['path'].values
test_labels = tf.keras.utils.to_categorical(test_df['label_idx'].values, num_classes=len(label_to_index))

train_ds = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
train_ds = train_ds.map(lambda x, y: preprocess_image(x, y)).batch(BATCH_SIZE).shuffle(1000)

test_ds = tf.data.Dataset.from_tensor_slices((test_paths, test_labels))
test_ds = test_ds.map(lambda x, y: preprocess_image(x, y)).batch(BATCH_SIZE)

print("دیتاست‌های نهایی")

# مرحله ۵: لود مدل 1 و استخراج ویژگی‌ها
model_view = load_model("view_classifier(s2-model1).keras")
feature_extractor_view = Sequential(model_view.layers[:-1])

def extract_features(dataset):
    X_img, X_view, Y = [], [], []
    for batch_imgs, batch_labels in dataset:
        feats = feature_extractor_view(batch_imgs, training=False)
        X_img.append(batch_imgs.numpy())
        X_view.append(feats.numpy())
        Y.append(batch_labels.numpy())
    return (np.vstack(X_img), np.vstack(X_view), np.vstack(Y))

X_train_img, X_train_view, y_train = extract_features(train_ds)
X_test_img, X_test_view, y_test = extract_features(test_ds)

print(" Feature extraction کامل شد.")
print("Train:", X_train_img.shape, X_train_view.shape, y_train.shape)
print("Test:", X_test_img.shape, X_test_view.shape, y_test.shape)

In [ ]:
import random
random_indices = random.sample(range(len(train_paths)), 10)

fig, axes = plt.subplots(2, 5, figsize=(15, 8))
axes = axes.ravel()

for i, idx in enumerate(random_indices):
    img_path = train_paths[idx]
    img = Image.open(img_path)
    img = img.resize((224, 224))
    axes[i].imshow(img)
    axes[i].axis('off')
    axes[i].set_title(f"Label: {train_labels[idx]}", fontsize=10)

plt.tight_layout()
plt.show()


In [ ]:
feature_extractor_view.summary()


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Concatenate
from tensorflow.keras.optimizers import Adam

IMAGE_SIZE = 128

# Base model (VGG16)
base_model = VGG16(
    input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
    include_top=False,
    weights="imagenet"
)

for layer in base_model.layers:
    layer.trainable = False

for layer in base_model.layers[-4:]:
    layer.trainable = True

# ساخت شاخه تصویر
img_input = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3), name="image_input")
x = base_model(img_input, training=False)
x = Flatten()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)

# ساخت شاخه ویژگی نما
view_input = Input(shape=(128,), name="view_input")
y = Dense(64, activation='relu')(view_input)

# ترکیب دو شاخه
combined = Concatenate()([x, y])
z = Dropout(0.3)(combined)
z = Dense(128, activation='relu')(z)
z = Dropout(0.2)(z)

num_classes = y_train.shape[1]
output = Dense(num_classes, activation="softmax")(z)

# مدل نهایی
final_model = Model(inputs=[img_input, view_input], outputs=output)

final_model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"])

final_model.summary()


In [ ]:
history = final_model.fit(
    [X_train_img, X_train_view], y_train,
    validation_data=([X_test_img, X_test_view], y_test),
    epochs=5,
    batch_size=32)


In [ ]:
from tensorflow.keras.models import load_model
final_model.save('final-model+pre-s2.keras')

#**Classification** **Report**

In [ ]:
test_predictions = final_model.predict([X_test_img, X_test_view])
y_pred = np.argmax(test_predictions, axis=1)
y_true = np.argmax(y_test, axis=1)

from sklearn.metrics import classification_report, confusion_matrix
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=[str(c) for c in range(y_test.shape[1])]))

cm = confusion_matrix(y_true, y_pred)

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=[str(c) for c in range(y_test.shape[1])],
            yticklabels=[str(c) for c in range(y_test.shape[1])])
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()


#**محاسبه ROC و AU**

In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

y_true_bin = label_binarize(y_true, classes=list(range(y_test.shape[1])))
n_classes = y_test.shape[1]

plt.figure(figsize=(8,6))
for i in range(n_classes):
    fpr, tpr, _ = roc_curve(y_true_bin[:, i], test_predictions[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"Class {i} (AUC = {roc_auc:.2f})")

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curves (One-vs-Rest)")
plt.legend(loc="lower right")
plt.show()


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_dir = "/content/drive/MyDrive/archive/Testing"
IMAGE_SIZE = 128
BATCH_SIZE = 32
test_datagen = ImageDataGenerator(rescale=1./255)
test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False)


In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import classification_report

tumor_classes = list(test_gen.class_indices.keys())
view_classes = ["axial", "coronal", "sagittal"]
IMAGE_SIZE = 128

y_pred = np.argmax(final_model.predict([X_test_img, X_test_view]), axis=1)
y_true = np.argmax(y_test, axis=1)

report = classification_report(y_true, y_pred, target_names=tumor_classes, output_dict=True)

f1_dict = {cls: report[cls]['f1-score'] for cls in tumor_classes}
precision_dict = {cls: report[cls]['precision'] for cls in tumor_classes}
recall_dict = {cls: report[cls]['recall'] for cls in tumor_classes}

print(" Performance metrics calculated for each tumor class.")

def detect_tumor_and_view(img_path, final_model, feature_extractor_view, model_view, image_size=IMAGE_SIZE):

    try:
        img = load_img(img_path, target_size=(image_size, image_size))
        img_array = img_to_array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)

        view_pred = model_view.predict(img_array)
        view_class_idx = np.argmax(view_pred, axis=1)[0]
        view_label = view_classes[view_class_idx]
        view_feature = feature_extractor_view(img_array, training=False).numpy()

        tumor_pred = final_model.predict([img_array, view_feature])
        tumor_class_idx = np.argmax(tumor_pred, axis=1)[0]
        tumor_label = tumor_classes[tumor_class_idx]

        f1 = f1_dict.get(tumor_label, 0)
        prec = precision_dict.get(tumor_label, 0)
        rec = recall_dict.get(tumor_label, 0)

        if "notumor" in tumor_classes and tumor_label == "notumor":
            result = f"No Tumor detected (Precision: {prec*100:.1f}%, Recall: {rec*100:.1f}%, F1: {f1*100:.1f}%)\nView: {view_label}"
        else:
            result = f"Tumor: {tumor_label}\n(Precision: {prec*100:.1f}%, Recall: {rec*100:.1f}%, F1: {f1*100:.1f}%)\nView: {view_label}"

        plt.imshow(load_img(img_path))
        plt.axis("off")
        plt.title(result)
        plt.show()

        return tumor_label, view_label, f1, prec, rec

    except Exception as e:
        print(" Error processing the image:", str(e))
        return None


In [ ]:
result = detect_tumor_and_view(
    "/content/drive/MyDrive/h-archive/Testing/glioma1-sagittal/Te-gl_0161.jpg",
    final_model, feature_extractor_view, model_view)
print("Prediction Result:", result)
